# `OscaRobber`

# Part I: Data preparation

In this section we decide on libraries and data structures we will use across the notebook, but also perform the initial preprocessing of the data to a shape which is best for use in further analysis. We'll show how to 
* Fetch data from IMDB website
* Convert plaintext files to SQLite tables
* Bring structure into the data by preprocessing it using bunch of helper classes
* Import it to MongoDB and make access super-fast

## How to fetch IMDB data

First of all we start by fetching the data. There are two different ways to fetch the data from the IMDB website.
* One way, fetching information through the http://akas.imdb.com web server by using the http data access system. This can be a problem, because it means that a lot of web pages must be fetched and parsed, and this can be time and bandwidth consuming, and before we get very far in a big data query we could see our IP address blocked.
* To help serve users who want local access to their data, however, the IMDB offers their data set as plain text files that can be downloaded for free. Thus, another way, using a local SQL (here SQLite3) database, after we've populated it with the data from the plain text data files (we can have a local copy of the database from http://www.imdb.com/interfaces/) using the `imdbpy2sql.py` script. 

We choose the local approach: 
* Selected a mirror of the "The Plain Text Data Files" from the http://www.imdb.com/interfaces/ page
* Retrieved 1.100 files from IMDB's FTP-server, taking 12,54 GB on disk space
* Created a database named "imdb" using SQLite3:

`sqlite3 imdb`

`> create table emptyTable(field1 int); drop table emptyTable;`

* Created the tables and populated the database by running the imdbpy2sql.py script:

`# imdbpy2sql.py -d /dir/with/plainTextDataFiles/ -u 'sqlite:///full/path/to/database'`

The `imdbpy2sql.py` script has a number of command line arguments, useful to chose amongst presets that can improve performances, using specific database servers. We used the `--sqlite-transactions` command
line option to obtain acceptable performances. SQLite seems to hugely benefit from the use of a non-journaling
filesystem and/or of a ramdisk/tmpfs.

The time elapsed:

`# TIME TOTAL TIME TO INSERT/WRITE DATA : 72min, 40sec (wall) 60min, 18sec (user) 3min, 15sec (system)`

`# TIME FINAL : 84min, 50sec (wall) 68min, 22sec (user) 4min, 11sec (system)`

**The final DB takes 9,76 GB on disk.**

## SQLite? Not good

Using SQLite seems straightforward and quite obvious (e.g., relational databases perform quite good by joins). But there are downsides. First of all, many columns have to be parsed with regular expressions in order to extract a desirable attribute. This can't be done with `sqlite3` library itself. Second, SQLite doesn't allow to group values without aggregating, i.e., we cannot put attribute values of the same id into a list or a set. Third, complex mapping isn't possible. And finally, clause "distinct" to drop duplicates works 10 times slower than by using native Python. For all that reasons, we will transform the data outside of SQLite and store it into MongoDB, as it allows storing lists, fast indexing of ids, and more. We will get a bunch of collections in MongoDB which can be queried by movie id, person id, or similar, and return values of the corresponding attribute. Those values will be clean and ready for use in all analysis steps later on.

## Which libraries to use 

There are many options for processing the raw data from SQLite and transforming it in a way that we can use it in further steps of our analysis. 

#### 1. Option: Using `blaze`

> The Blaze Ecosystem provides Python users high-level access to efficient computation on inconveniently large data. 

We tried to use Blaze in the project to manipulate data, but the performance was ugly: 
* SQLite queries were executed very inefficiently
* Transforming SQLAlchemy object to `pandas.DataFrame` lasted infinite time

For example peace of code below performed in **86.54 sec**.

While using native `pandas` performed in **5.52 sec**.

Thus, Blaze is not a right candidate to achieve the best performance in transforming data and transferring it from one system into another.

#### 2. Option: Using `sqlite3` and `pickle`

Using pickle is a terrible idea that should be avoided whenever possible:
* Pickle is both slower and produces larger serialized values than most of the alternatives
* Another reason not to use pickle is that unpickling malicious data can cause security issues, including arbitrary code execution.
* For each update you will have to remember to pickle the entire data and save it to disk.

#### 3. Option: Using `sqlite3`, `pandas` and `pymongo` 

This option performs best, and without any overheads produced by other third-party libraries like `blaze`. We can take full control of each step and produce a very clean pipeline. Additionaly, we can get stats on every operation we undertake, so it makes the code analysis a lot easier than with comparable solutions.

## How to transform the data

A set of helpers needs to be defined in order to process data transparently and in a matter we want.

***Important: We will rely on `pandas.DataFrame` data structure to represent data retrieved from SQLite and work on it. An option would be to work on lists, but it would also mean a lot of additional code which we want to avoid. So every helper defined below works mainly on dataframes.***

### Printing tables

Decision on how to print data from SQLite and MongoDB is not easy. There are many ways, from `json.dumps` to `PrettyPrinter`, but the coolest way is to define the class ourselfes. The `Table` class, as other helpes, will base upon `pandas.DataFrame` functionality as it allows us to print a tabular data nicely.

In [2]:
from lib.table import Table

Here a small example how the `Table` class works.

In [3]:
t = Table()
t.from_dict({'h1': [1, 2, 3], 'h2': [5, 4, 4], 'h3': [9, 1, 2]})
t.sort_values(by=['h2', 'h3'], ascending=[True, False])
t.display()
# or
t.from_tuples([(1, 5, 9), (2, 4, 1), (3, 4, 2)], columns=['h1', 'h2', 'h3'])
t.sort_values(by=['h2', 'h3'], ascending=[True, False])
t.display()


   h1  h2  h3
2   3   4   2
1   2   4   1
0   1   5   9


   h1  h2  h3
2   3   4   2
1   2   4   1
0   1   5   9



### Fetching and processing SQLite data

Here we define a SQLite helper, which encapsulates a pipeline for preprocessing the data. Working on raw tables (means analyzing them without preprocessing) is a headshot, as many columns require text extraction using regular expressions, but also cleaning, grouping and format conversion. This all is done by a single pipeline consisting of following steps:
1. Import data from SQLite into a dataframe
2. Drop duplicates
3. If some column contains joined text data or needs to be parsed using a regular expression, then extract texts (optional)
4. Apply map over a parsed column (optional, e.g., convert string to int)
5. Group rows by id and aggregate them into a list (handling one-to-many relationsships)
6. Perform reduce operation on lists (optional, e.g., compute the max of the list)

It's implemented using knowledge about patterns in the data and with regard to further transmission to MongoDB.

Here is an example of preprocessing movie gross data gathered from SQLite. IMDB tables often contain records like below: different attributes are stored under different type ids. Our goal is to split those attributes (perform filter), save them to new dataframes and apply all tools on each of them.

    1. Read SQL
First we execute a SQL statement on SQLite database to get desired records and save them in a dataframe

| id | type | info |
|-|-|-|
| 3131991 | 4 | "English" |
| 3131991 | 8 | "USA" |
| 3131991 | 107 | "\$421,000" |
| 3131991 | 107 | "\$421,000" |
| 3131991 | 107 | "\$581,000" |

    2. Drop duplicates
Then we take care of duplicate records

| id | type | info |
|-|-|-|
| 3131991 | 4 | "English" |
| 3131991 | 8 | "USA" |
| 3131991 | 107 | "\$421,000" |
| 3131991 | 107 | "\$581,000" |

    3. Filter
Split attributes by value of some third-party column

| id | type | languages |
|-|-|-|
| 3131991 | 4 | "English" |

| id | type | prod_countries |
|-|-|-|
| 3131991 | 8 | "USA" |

| id | type | gross |
|-|-|-|
| 3131991 | 107 | "\$421,000" |
| 3131991 | 107 | "\$581,000" |

    4. Remove column
Remove column, i.e., the third-party column used to split attributes

| id | languages |
|-|-|
| 3131991 | "English" |

| id | prod_countries |
|-|-|
| 3131991 | "USA" |

| id | gross |
|-|-|
| 3131991 | "\$421,000" |
| 3131991 | "\$581,000" |

    5. Extract text
Now let's take a look at gross. We want not strings in gross column but integers, so remove dollar sign along with commas

| id | gross |
|-|-|
| 3131991 | "421000" |
| 3131991 | "581000" |

    6. Map
Convert strings to integers

| id | gross |
|-|-|
| 3131991 | 421000 |
| 3131991 | 581000 |

    7. Group
Joins in IMDB database can be performed on tables with either one-to-one or one-to-many relationship, thus some of ids will have one value, some more. Group them all into a list. If an id has only one value assigned, put this value into a list so the whole column is of one data type.

| id | gross |
|-|-|
| 3131991 | [421000, 581000] |

    8. Reduce
In IMDB database, gross records are cumulative over time, so we need to choose only the latest record (i.e. biggest in value), thus reduce data using `max`

| id | gross |
|-|-|
| 3131991 | [581000] |

Take notice that the result of the group operation is a list, not a scalar. This is wanted. On some dataframes we want to perform reduce, on some not, thus some of them will contain single values, some lists, when returned. By having diverse data structures, we would need to implement additional helpers to work on them. If you don't understand what I say - just trust me.

***Hence every item in a dataframe will be a list (except id).***

    9. Encode
Some of the attributes will contain strings reoccurring in many rows. Those strings take more place than if we were working with integers. Let's take a small example of how "Denmark" of the attribute "production country" compares with its encoded version.

In [4]:
from pympler.asizeof import asizeof
print '"Denmark" as string: %d' % asizeof('Denmark')
print '"Denmark" as integer: %d' % asizeof(154)

"Denmark" as string: 48
"Denmark" as integer: 24


Thus for each of those attributes we build a map and encode the attribute according to that map. We build a map by extracting all unique values and enumerating them. By doing this we get two different dataframes: 
* Dataframe containing an integer-encoded attribute
* Dataframe containing map of this attribute to get initial string values later on

For example:

| id | genres |
|-|-|
| 3131991 | ["Action", "Comedy"] |

By encoding document we get encoded dataframe

| id | genres |
|-|-|
| 3131991 | [23, 56] |

and a map

| id | genres\_map |
|-|-|
| 23 | ["Action"] |
| 56 | ["Comedy"] |

***This way we save a lot of space.***

In [5]:
from lib.litehelper import LiteHelper
lite = LiteHelper('/Users/olegpolakow/Documents/imdb_data/imdb.sqlite')

### Pushing data to MongoDB

MongoDB helper makes transmitting data to MongoDB a lot easier. It does:
1. Take a dataframe as input
2. Extract each column from the dataframe as `pandas.Series` object
3. Convert series into a format accepted by MongoDB (e.g., list of dicts)
4. Store documents in a new collection which name is a column name in the dataframe
5. Create an index so document retrieval is super-fast

It also shows statistics on collections. 

In [6]:
from lib.mongohelper import MongoHelper
mongo = MongoHelper('imdb')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'imdb')


### External libraries

In [7]:
# We need np.nan to fill missing data
import numpy as np
import re

## Begin data processing

We have 3 different sources of data:

* IMDB data in SQLite
* User reviews
* JSON files obtained from OMDB service

All of them will be processed in distinct sections of this notebook and exported as collections to MongoDB.

## IMDB data

Now it's time to fetch the data from SQLite, preprocess it, and transfer to MongoDB. 

***As already mentioned before, many collections will contain ids rather than strings or values (like keyword_ids instead of keywords) to save some space, as integers take less space than strings, so we will need to define maps (collections ending by "_map") to get initial values. Don't be scared if you see something like that, it's actually pretty awesome!***

For example:

| movie_id | keyword_id |
|-|-|
| 123 | 7834 |

using

| keyword_id | keyword |
|-|-|
| 7834 | love |

translates into

| movie_id | keyword |
|-|-|
| 123 | love |

### Two sets of attributes

Our movie-actor network will be bipartite, thus we will work on both its projections: movie network projection and person network projection. 
* Attributes applied on nodes of movie network projection are movie attributes, like release years, runtimes, and more (see below). We will prepare them first. 
* Afterwards, we will prepare person attributes like birth place and year, which will be attached to nodes of person network projection.

### Movie attributes

We will create a bunch of collections with movie attributes. Some of them will be used as node attributes in network analysis, some will be used for similarity and sentiment analysis, and some as variables of classifiers. 

Now take a look at collections we will transfer to MongoDB and their properties.

* In column "Collection" we specify how the target collection of documents in MongoDB will be called. 
* In column "Search by" we specify the indexed property (id) by which we search for an attribute. 
* In column "Result" we specify which attribute will be returned.
* In column "Type" we specify the return type.

Collections which return original attribute values
    
| Collection | Search by | Return | Type |
|-|-|-|
| `release_years` | `movie_id` | release years | list of `int` |
| `runtimes` | `movie_id` | runtimes | list of `int` |
| `plots` | `movie_id` | plots | list of `str` |
| `budgets` | `movie_id` | budgets | list of `int` |
| `gross` | `movie_id` | gross | list of `int` |
| `votes` | `movie_id` | number of votes | list of `int` |
| `imdb_ratings` | `movie_id` | imdb ratings | list of `float` |

Collections which return encoded attribute values (you need a map to decode)

| Collection | Search by | Return | Type |
|-|-|-|
| `genres` | `movie_id` | `genre_id` | list of `int` |
| `languages` | `movie_id` | `language_id` | list of `int` |
| `prod_countries` | `movie_id` | `prod_countries_id` | list of `int` |
| `keywords` | `movie_id` | `keyword_id` | list of `int` |
| `prod_companies` | `movie_id` | `prod_companies_id` | list of `int` |
| `cast` | `movie_id` | `person_id` | list of `int` |
| `directors` | `movie_id` | `person_id` | list of `int` |

Collections which are used to decode a specific attribute for an original value

| Collection | Search by | Result | Type |
|-|-|-|
| `titles_map` | `movie_id` | titles | list of `str` |
| `genres_map` | `genre_id` | genres | list of `str` |
| `languages_map` | `language_id` | languages | list of `str` |
| `prod_countries_map` | `prod_countries_id` | production countries | list of `str` |
| `keywords_map` | `keyword_id` | keywords | list of `str` |
| `prod_companies_map` | `company_id` | company names | list of `str` |

***Notice that results are always lists containing one or more values. This is done because we want to treat single (from an one-to-one relationship) as well as multiple values (from an one-to-many relationship) uniformly.***

### Table `title`

Following collections are stored:
* `titles_map`: Map of movie titles. All movie attributes are keyed by `movie_id`, but if you want a title for that `movie_id`, than look into this map. 
* `release_years`: Release year of a movie.

In [8]:
mongo.store(
    *lite.pipeline(
        '''    
            SELECT 
                title.id AS id, 
                title.title AS titles_map, 
                title.production_year AS release_years
            FROM title 
            WHERE title.kind_id = 1 
        ''',
        col_map={'year_info': lambda x: int(x) if x else np.nan}
    )
)

# SQLite

         Operation                           Column   Count  Time (sec)
0         Read SQL  [id, titles_map, release_years]  963014   13.590428
1  Drop duplicates  [id, titles_map, release_years]  933715    0.683886


  Operation                           Column   Count  Time (sec)
0     Group  [id, titles_map, release_years]  933715  152.196688

# MongoDB

           Operation     Collection   Count  Time (sec)
0  Convert dataframe     titles_map  933715    1.156781
1  Convert dataframe  release_years  933715    2.873072
2    Store documents     titles_map  933715   27.198547
3       Create index     titles_map            4.496047
4    Store documents  release_years  933715   57.472108
5       Create index  release_years            2.803487


     collections   count  size (MB)  storage (MB)
0     titles_map  933715         59            14
1  release_years  933715         46             0




In [9]:
mongo.collect('titles_map', filter={'id': 3734624})

{3734624: [u'The Dark Knight Rises']}

In [10]:
mongo.collect('release_years', filter={'id': 3734624})

{3734624: [2012.0]}

### Table `movie_info`

Following collections are stored:
* `runtimes`: Runtime of a movie given in minutes.
* `genres`: Genres of a movie. Encoded with `genres_map`.
* `languages`: Languages spoken in a movie. Encoded with `languages_map`.
* `countries`: Countries in which a movie was produced. Encoded with `prod_countries_map`.
* `plots`: Plots of a movie.
* `budgets`: Budget of a movie.
* `gross`: Gross of a movie.

In [11]:
mongo.store(
    *lite.pipeline(
        '''
            SELECT 
                title.id AS id, 
                movie_info.info_type_id AS type, 
                movie_info.info AS info
            FROM title
            INNER JOIN movie_info
            ON title.id = movie_info.movie_id
            WHERE title.kind_id = 1 
            AND movie_info.info_type_id IN (1, 3, 4, 8, 98, 105, 107)
        ''',
        col_filter={
            ('type', 1): {'info': 'runtimes'},
            ('type', 3): {'info': 'genres'},
            ('type', 4): {'info': 'languages'},
            ('type', 8): {'info': 'prod_countries'},
            ('type', 98): {'info': 'plots'},
            ('type', 105): {'info': 'budgets'},
            ('type', 107): {'info': 'gross'}
        },
        col_drop=['type'],
        col_extract={
            'runtimes': r'(?:.*:)?(?P<runtimes>\d*)',
            'budgets': r'\$(?P<budgets>[\d,]*)',
            'gross': r'\$(?P<gross>[\d,]*)'
        },
        col_map={
            'runtimes': lambda x: int(x) if x else np.nan,
            'budgets': lambda x: int(''.join(x.split(','))) if x else np.nan,
            'gross': lambda x: int(''.join(x.split(','))) if x else np.nan
        },
        col_reduce={
            'gross': lambda x: [max(x)]
        },
        col_encode={
            'genres': 'genres_map', 
            'languages': 'languages_map',
            'prod_countries': 'prod_countries_map'
        }
    )
)

# SQLite

         Operation                            Column    Count  Time (sec)
0         Read SQL                  [id, type, info]  4923514  173.320327
1  Drop duplicates                  [id, type, info]  4919819    2.659981
2           Filter       df[type==4] info->languages   909870    0.103121
3           Filter         df[type==107] info->gross   169922    0.039586
4           Filter       df[type==105] info->budgets   174597    0.026757
5           Filter          df[type==3] info->genres  1745934    0.176832
6           Filter  df[type==8] info->prod_countries  1023371    0.101818
7           Filter          df[type==98] info->plots   244105    0.034201
8           Filter        df[type==1] info->runtimes   652020    0.079657


  Operation           Column                                    Count  \
0     Group  [id, languages]                                   848085   
1    Encode        languages  848085 (languages), 342 (languages_map)   

   Time (sec)  
0   74.17120

In [12]:
mongo.collect('runtimes', filter={'id': 3734624})

{3734624: [164.0]}

In [13]:
mongo.collect('genres', filter={'id': 3734624}, applymap='genres_map')

{3734624: [u'Action', u'Thriller']}

In [14]:
mongo.collect('languages', filter={'id': 3734624}, applymap='languages_map')

{3734624: [u'English', u'Arabic']}

In [15]:
mongo.collect('prod_countries', filter={'id': 3734624}, applymap='prod_countries_map')

{3734624: [u'USA', u'UK']}

In [16]:
mongo.collect('plots', filter={'id': 3734624})

{3734624: [u"Despite his tarnished reputation after the events of The Dark Knight, in which he took the rap for Dent's crimes, Batman feels compelled to intervene to assist the city and its police force which is struggling to cope with Bane's plans to destroy the city.",
  u"8 years after the events of The Dark Knight find Gotham at a time of peace. This is due to Batman taking the fall for Harvey Dent's murder. However, a new evil force named Bane has arrived in Gotham and aims to take over the city and expose the truth behind who Harvey Dent really was. Now that Wayne Manor has been completely rebuilt - Bruce Wayne has become almost reclusive, rarely leaving the estate. And with Bane taking over the city by force, it forces Batman to come out of retirement. But his allies are few and far apart. An elusive jewel thief by the name of Selina Kyle could be the key to stopping Bane - but whose side is she on?",
  u"8 years after the Batman taking the fall for murders committed by Harvey, 

In [17]:
mongo.collect('budgets', filter={'id': 3734624})

{3734624: [250000000]}

In [18]:
mongo.collect('gross', filter={'id': 3734624})

{3734624: [1084939099]}

### Table `movie_info_ids`

* `votes`: Number of votes on IMDB website of a movie.
* `imdb_ratings`: Rating on IMDB website of a movie.

In [19]:
mongo.store(
    *lite.pipeline(
        '''
            SELECT 
                title.id AS id, 
                movie_info_idx.info_type_id AS type, 
                movie_info_idx.info AS info
            FROM title
            INNER JOIN movie_info_idx
            ON title.id = movie_info_idx.movie_id
            WHERE title.kind_id = 1 
            AND movie_info_idx.info_type_id in (100, 101)
        ''',
        col_filter={
            ('type', 100): {'info': 'votes'},
            ('type', 101): {'info': 'imdb_ratings'}
        },
        col_drop=['type'],
        col_extract={
            'votes': r'(?P<votes>\d*)', 
            'imdb_ratings': r'(?P<imdb_ratings>\d*(?:\.\d*)?)'
        },
        col_map={
            'votes': lambda x: int(x) if x else np.nan, 
            'imdb_ratings': lambda x: float(x) if x else np.nan
        }
    )
)

# SQLite

         Operation                            Column   Count  Time (sec)
0         Read SQL                  [id, type, info]  560888   30.364634
1  Drop duplicates                  [id, type, info]  560888    0.118375
2           Filter  df[type==101] info->imdb_ratings  280444    0.027502
3           Filter         df[type==100] info->votes  280444    0.022852


  Operation                      Column   Count  Time (sec)
0     Parse  imdb_ratings->imdb_ratings  280444    0.559324
1       Map                imdb_ratings  280444    0.196974
2     Group          [id, imdb_ratings]  280444   21.756898


  Operation        Column   Count  Time (sec)
0     Parse  votes->votes  280444    0.730650
1       Map         votes  280444    0.344960
2     Group   [id, votes]  280444   21.685771

# MongoDB

           Operation    Collection   Count  Time (sec)
0  Convert dataframe  imdb_ratings  280444    0.387501
1    Store documents  imdb_ratings  280444    8.555460
2       Create index

In [20]:
mongo.collect('votes', filter={'id': 3734624})

{3734624: [1165380]}

In [21]:
mongo.collect('imdb_ratings', filter={'id': 3734624})

{3734624: [8.5]}

### Table `movie_keyword`

* `keywords`: Keywords of a movie. Encoded with `keywords_map`.

In [22]:
mongo.store(
    *lite.pipeline(
        '''
            SELECT 
                title.id AS id, 
                keyword.keyword AS keywords
            FROM title
            INNER JOIN movie_keyword
            ON title.id = movie_keyword.movie_id
            INNER JOIN keyword
            ON movie_keyword.keyword_id = keyword.id
            WHERE title.kind_id = 1
        ''',
        col_encode={
            'keywords': 'keywords_map'
        }
    )
)

# SQLite

         Operation          Column    Count  Time (sec)
0         Read SQL  [id, keywords]  3758557   78.341310
1  Drop duplicates  [id, keywords]  3758557    2.304475


  Operation          Column                                     Count  \
0     Group  [id, keywords]                                    267438   
1    Encode        keywords  267438 (keywords), 147421 (keywords_map)   

   Time (sec)  
0   24.852768  
1    4.601341  

# MongoDB

           Operation    Collection   Count  Time (sec)
0  Convert dataframe  keywords_map  147421    0.297392
1    Store documents  keywords_map  147421    5.193866
2       Create index  keywords_map            0.477687


    collections   count  size (MB)  storage (MB)
0  keywords_map  147421          9             0



           Operation Collection   Count  Time (sec)
0  Convert dataframe   keywords  267438    0.423758
1    Store documents   keywords  267438   12.722620
2       Create index   keywords            0.929561


  colle

In [23]:
mongo.collect('keywords', filter={'id': 3734624}, applymap='keywords_map')

{3734624: [u'2010s',
  u'action-hero',
  u'action-heroine',
  u'aerial-shot',
  u'aircraft',
  u'airplane',
  u'airplane-accident',
  u'airplane-hijacking',
  u'airport',
  u'ak-47',
  u'altered-version-of-studio-logo',
  u'ambulance',
  u'ambush',
  u'american-abroad',
  u'american-flag',
  u'american-football',
  u'analgesic',
  u'anti-hero',
  u'anti-heroine',
  u'apartment',
  u'appeared-on-tv-news',
  u'apple',
  u'armored-car',
  u'armorer',
  u'armory',
  u'army',
  u'assassin',
  u'assassination-attempt',
  u'attempted-robbery',
  u'bag-over-head',
  u'ball',
  u'bane-the-character',
  u'bankruptcy',
  u'bar',
  u'bare-chested-male',
  u'barefoot',
  u'based-on-comic',
  u'based-on-comic-book',
  u'bat',
  u'bat-cave',
  u'battle',
  u'beaten-to-death',
  u'beating',
  u'betrayal',
  u'billionaire',
  u'blindness',
  u'blockade',
  u'blockbuster',
  u'blood-transfusion',
  u'bloody-body-of-child',
  u'boardroom',
  u'body-bag',
  u'bodyguard',
  u'bomb',
  u'booby-trap',
  u'bo

### Table `movie_companies`

* `prod_companies`: Production companies of a movie. Encoded with `prod_companies_map`.

In [24]:
mongo.store(
    *lite.pipeline(
        '''
            SELECT 
                title.id AS id,
                company_name.name AS prod_companies
            FROM title
            INNER JOIN movie_companies
            ON title.id = movie_companies.movie_id
            INNER JOIN company_name
            ON movie_companies.company_id = company_name.id
            WHERE title.kind_id = 1 
            AND movie_companies.company_type_id = 2
        ''',
        col_encode={
            'prod_companies': 'prod_companies_map'
        }
    )
)

# SQLite

         Operation                Column   Count  Time (sec)
0         Read SQL  [id, prod_companies]  874878   41.251127
1  Drop duplicates  [id, prod_companies]  872175    0.440080


  Operation                Column  \
0     Group  [id, prod_companies]   
1    Encode        prod_companies   

                                               Count  Time (sec)  
0                                             634875   56.416439  
1  634875 (prod_companies), 222116 (prod_companie...    2.437642  

# MongoDB

           Operation          Collection   Count  Time (sec)
0  Convert dataframe  prod_companies_map  222116    0.355735
1    Store documents  prod_companies_map  222116    7.648263
2       Create index  prod_companies_map            0.625628


          collections   count  size (MB)  storage (MB)
0  prod_companies_map  222116         14             0



           Operation      Collection   Count  Time (sec)
0  Convert dataframe  prod_companies  634875    0.834945
1    St

In [25]:
mongo.collect('prod_companies', filter={'id': 3734624}, applymap='prod_companies_map')

{3734624: [u'DC Comics',
  u'DC Entertainment',
  u'Legendary Entertainment',
  u'Syncopy',
  u'Warner Bros.']}

### Table `cast_info`

* `cast`: Actors of a movie. Encoded with `names_map`. 
* `directors`: Movie directors of a movie. Encoded with `names_map`.

In [26]:
mongo.store(
    *lite.pipeline(
        '''
            SELECT title.id AS id, cast_info.person_id AS cast
            FROM title
            INNER JOIN cast_info
            ON title.id = cast_info.movie_id
            WHERE title.kind_id = 1 
            AND cast_info.role_id IN (1, 2)
        ''',
        col_reduce={
            'cast': lambda x: x[:5] if len(x)>5 else x
        },
    )
)
mongo.store(
    *lite.pipeline(
        '''
            SELECT title.id AS id, cast_info.person_id AS directors
            FROM title
            INNER JOIN cast_info
            ON title.id = cast_info.movie_id
            WHERE title.kind_id = 1 
            AND cast_info.role_id = 8
        '''
    )
)

# SQLite

         Operation      Column    Count  Time (sec)
0         Read SQL  [id, cast]  7608461  737.768490
1  Drop duplicates  [id, cast]  7503299    2.477453


  Operation      Column   Count  Time (sec)
0     Group  [id, cast]  725011   60.536254
1    Reduce        cast  725011    0.847770

# MongoDB

           Operation Collection   Count  Time (sec)
0  Convert dataframe       cast  725011    1.558348
1    Store documents       cast  725011   27.570685
2       Create index       cast            2.495254


  collections   count  size (MB)  storage (MB)
0        cast  725011         47             0


# SQLite

         Operation           Column   Count  Time (sec)
0         Read SQL  [id, directors]  945590   63.478911
1  Drop duplicates  [id, directors]  944038    0.269218


  Operation           Column   Count  Time (sec)
0     Group  [id, directors]  853946   66.910929

# MongoDB

           Operation Collection   Count  Time (sec)
0  Convert dataframe  directors  853946 

In [27]:
mongo.collect('cast', filter={'id': 3734624}, applymap='names_map')

{3734624: [u'Abbink, Marc',
  u'Abbott, Joseph M.',
  u'Aboutboul, Alon',
  u'Adams, Isiah',
  u'Adkins, Donnie']}

In [28]:
mongo.collect('directors', filter={'id': 3734624}, applymap='names_map')

{3734624: [u'Nolan, Christopher']}

### Person attributes

Collections below contain person attributes. Those attributes will be applied to nodes of person network projection.

Collections with original attribute values
    
| Collection | Sea | Result | Type | 
|-|-|-|
| `birth_years` | `person_id` | birth years | list of `int` |

Collections with encoded attribute values

| Collection | Search by | Result | Type | 
|-|-|-|
| `birth_places` | `person_id` | `birth_place_id` | list of `int` |

Collections for decoding encoded attribute values

| Collection | Search by | Result | Type |
|-|-|-|
| `names_map` | `person_id` | names | list of `str` |
| `birth_places_map` | `birth_place_id` | birth places | list of `str` |

### Table `name`

Map of names of actors and directors of all movies. Unlike previous collections, names are keyed by `person_ids`, as we store person attributes.

In [29]:
mongo.store(
    *lite.pipeline(
        '''
            SELECT name.id as id, name as names_map
            FROM name
            INNER JOIN cast_info
            ON name.id = cast_info.person_id
            INNER JOIN title
            ON cast_info.movie_id = title.id
            WHERE title.kind_id = 1
            AND (cast_info.role_id IN (1, 2) OR cast_info.role_id = 8)
        '''
    )
)

# SQLite

         Operation           Column    Count   Time (sec)
0         Read SQL  [id, names_map]  8554051  1081.075714
1  Drop duplicates  [id, names_map]  2851082     4.111867


  Operation           Column    Count  Time (sec)
0     Group  [id, names_map]  2851082  251.778594

# MongoDB

           Operation Collection    Count  Time (sec)
0  Convert dataframe  names_map  2851082    4.371699
1    Store documents  names_map  2851082   88.042715
2       Create index  names_map             9.467836


  collections    count  size (MB)  storage (MB)
0   names_map  2851082        172            59




### Table `person_info`

* `birth_places`: Birth place of a person. Encoded with `birth_places_map`.
* `birth_years`: Birth year of a person.
* `heights`: Height of a person in cm.

In [30]:
def inches_to_cm(inches_str):
    foots_res = re.search(r'(.*)\'', inches_str)
    if foots_res:
        if foots_res.group(1).isdigit():
            foots = int(foots_res.group(1))
            inches_res = re.search(r'\'\s*(\d*)(\s+\d*\/\d*)?\"', inches_str)
            inches = 0
            if inches_res:
                if inches_res.group(1):
                    inches += int(inches_res.group(1))
                if inches_res.group(2):
                    inches += float(eval(inches_res.group(2)+'.0'))
            return int(foots * 30.48 + inches * 2.54)
    return np.nan
def parse_height(height_str):
    if height_str:
        search = re.search(r'(\d*)\s*cm', height_str)
        if search:
            if search.group(1):
                return int(search.group(1))
        else:
            return inches_to_cm(height_str)
    return np.nan
mongo.store(
    *lite.pipeline(
        '''
            SELECT 
                cast_info.person_id AS id, 
                person_info.info_type_id AS type, 
                person_info.info AS info
            FROM title
            INNER JOIN cast_info
            ON title.id = cast_info.movie_id
            INNER JOIN person_info
            ON cast_info.person_id = person_info.person_id
            WHERE title.kind_id = 1
            AND (cast_info.role_id IN (1, 2) OR cast_info.role_id = 8)
            AND person_info.info_type_id in (20, 21, 22)
        ''',
        col_filter={
            ('type', 20): {'info': 'birth_places'},
            ('type', 21): {'info': 'birth_years'},
            ('type', 22): {'info': 'heights'}
        },
        col_drop=['type'],
        col_extract={
            'birth_places': r'\s*(?P<birth_places>[\w]+)$', 
            'birth_years': r'(?P<birth_years>\d{4})$'
        },
        col_map={
            'birth_years': lambda x: int(x) if x else np.nan,
            'heights': parse_height
        },
        col_encode={
            'birth_places': 'birth_places_map'
        }
    )
)

# SQLite

         Operation                           Column    Count   Time (sec)
0         Read SQL                 [id, type, info]  8277731  1369.379506
1  Drop duplicates                 [id, type, info]   691298     1.935623
2           Filter   df[type==21] info->birth_years   279025     0.060522
3           Filter  df[type==20] info->birth_places   262604     0.049687
4           Filter       df[type==22] info->heights   149669     0.029224


  Operation                    Column   Count  Time (sec)
0     Parse  birth_years->birth_years  279025    0.831655
1       Map               birth_years  278288    0.413513
2     Group         [id, birth_years]  278288   22.006580


  Operation                      Column  \
0     Parse  birth_places->birth_places   
1     Group          [id, birth_places]   
2    Encode                birth_places   

                                           Count  Time (sec)  
0                                         262604    1.528038  
1          

In [31]:
mongo.collect('birth_places', filter={'id': 1615125}, applymap='birth_places_map')

{1615125: [u'UK']}

In [32]:
mongo.collect('birth_years', filter={'id': 1615125})

{1615125: [1970]}

In [33]:
mongo.collect('heights', filter={'id': 1615125})

{1615125: [181.0]}

## Import of user reviews

**Large Movie Review Dataset v1.0** fetched from http://ai.stanford.edu/~amaas/data/sentiment/

> The core dataset contains 50,000 reviews split evenly into 25k train and 25k test sets. The overall distribution of labels is balanced (25k pos and 25k neg). We also include an additional 50,000 unlabeled documents for unsupervised learning. In the entire collection, no more than 30 reviews are allowed for any given movie because reviews for the same movie tend to have correlated ratings.

The dataset provides unclassified as well as classified (by sentiment) user reviews. We used unlabeled data, as we are not interested in already provided sentiments, but want to calculate sentiments on ourselfes. 

Within the [train/unsup/] directory, reviews are stored in text files named following the convention [[id]\_[rating].txt] where [id] is a unique id and [rating] is the star rating for that review on a 1-10 scale. The [train/unsup/] directory has 0 for all ratings because the ratings are omitted for this portion of the dataset. For example, the file [train/unsup/200\_0.txt] is the text for a test set example with unique id 200, which corresponds to the line 200 in [urls\_[pos, neg, unsup].txt] file. This way we are able to connect reviews and IMDB ids.


All reviews are imported into one single collection. User reviews are keyed by IMDB ids rather than `movie_id`. 
    
| Collection | Search by | Result | 
|-|-|
| `user_reviews` | `imdb_id` | reviews |

In [34]:
from lib.reviewsparser import ReviewsParser
revparser = ReviewsParser(
    '/Users/olegpolakow/Documents/imdb_data/reviews/files', 
    '/Users/olegpolakow/Documents/imdb_data/reviews/urls.txt'
)

In [41]:
mongo.store(revparser.parse())

# ReviewsParser

              Operation  Count  Time (sec)
0             Parse ids  50000    0.141838
1           Parse paths  50000    0.568468
2                  Join   7091    3.991965
3  Convert to dataframe   7091    0.001628

# MongoDB

           Operation    Collection Count  Time (sec)
0  Convert dataframe  user_reviews  7091    0.005638
1    Store documents  user_reviews  7091    3.031173
2       Create index  user_reviews          0.100755


    collections  count  size (MB)  storage (MB)
0  user_reviews   7091         64             0




## Summary

In [42]:
mongo.stats(total=True)


           collections    count  size (MB)  storage (MB)
2            names_map  2851082        172            80
22      prod_countries   951419         44            18
9        release_years   933715         46            14
4           titles_map   933715         59            28
19              genres   886236         46            22
7            directors   853946         39            19
5            languages   848085         39            13
6                 cast   725011         47            29
16      prod_companies   634875         30            14
18            runtimes   613963         30            14
20               votes   280444         12             6
0         imdb_ratings   280444         13             6
8          birth_years   278288         12             6
14            keywords   267438         38            29
23        birth_places   250989         11             4
15  prod_companies_map   222116         14             7
17               plots   21834

We see that using filtering and intelligent mapping we were able to shrink the total size of IMDB data from 12 GB to 1 GB. 